# Librerías

Importamos las librerías necesarias y cargamos variables de entorno

In [21]:
%pip install pymongo
%pip install redis
%pip install neo4j
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from dotenv import load_dotenv
import os

load_dotenv(ROOT / ".env.local")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
REDIS_HOST = os.getenv("REDIS_HOST")
REDIS_PORT = os.getenv("REDIS_PORT")
REDIS_DB = os.getenv("REDIS_DB")
MONGO_URI = os.getenv("MONGO_URI")
MONGO_DB = os.getenv("MONGO_DB")

In [3]:
from apis.neo4j_api import Neo4jAPI
from apis.redis_api import RedisAPI
from apis.mongodb_api import MongoAPI
from apis.messaging_service import MessagingService
from apis.snapshot_service import SnapshotService
from apis.messageData import MessageData
from datetime import datetime
from apis.relationData import make_rel_id

neo4j_api = Neo4jAPI(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
redis_api = RedisAPI(REDIS_HOST, REDIS_PORT, REDIS_DB)
mongo_api = MongoAPI(MONGO_URI, MONGO_DB)
messaging_service = MessagingService(neo4j_api, redis_api, mongo_api)
snapshot_service = SnapshotService(mongo_api, neo4j_api, redis_api)

In [4]:
neo4j_api.clear_graph()
redis_api.clear_all_queues()
mongo_api.clear_messages()
mongo_api.clear_snapshots()

0

# Usuarios

In [5]:
users = [
    ("u1", "Alice"),
    ("u2", "Bob"),
    ("u3", "Charlie"),
    ("u4", "Diana"),
    ("u5", "Eva"),
    ("u6", "Frank"),
    ("u7", "Grace"),
    ("u8", "Hector"),
    ("u9", "Irene"),
    ("u10", "John")
]

for uid, name in users:
    try:
        neo4j_api.create_user(uid, name=name)
    except ValueError:
        pass # por si el usuario ya existe

relations = [
    ("u1", "u2"),
    ("u2", "u3"),
    ("u3", "u4"),
    ("u4", "u5"),
    ("u5", "u6"),
    ("u6", "u7"),
    ("u7", "u8"),
    ("u8", "u9"),
    ("u9", "u10"),
    ("u1", "u5")
]

for src, dst in relations:
    try:
        neo4j_api.create_relation(src, dst)
    except ValueError:
        pass # por si la relación ya existe

# Messaging service
API encargada del envío de mensajes entre usuarios. Realiza la gestión del envío y consumo de mensajes entre todas las bases de datos

# Enviar Mensaje
send_message(sender_id, receiver_id, content, metadata) encola el mensaje en la cola de redis correspondiente y aumenta el contador de mensajes en la relación correspondiente. Si la relación no existe, lanza excepción

In [6]:
messaging_service.send_message("u1", "u2", "Hola u2")
messaging_service.send_message("u1", "u2", "¿Qué tal?")
messaging_service.send_message("u1", "u2", "Mensaje 3")

messaging_service.send_message("u2", "u3", "Hola u3")
messaging_service.send_message("u2", "u3", "Seguimos la cadena")

messaging_service.send_message("u3", "u4", "Hola u4")
messaging_service.send_message("u3", "u4", "Otro mensaje")

messaging_service.send_message("u4", "u5", "Hola u5")
messaging_service.send_message("u4", "u5", "Mensaje extra")
messaging_service.send_message("u4", "u5", "Mensaje extra 2")

messaging_service.send_message("u1", "u5", "Atajo 1")
messaging_service.send_message("u1", "u5", "Atajo 2")

messaging_service.send_message("u5", "u6", "Hola u6")
messaging_service.send_message("u5", "u6", "Mensaje adicional")

messaging_service.send_message("u6", "u7", "Hola u7")
messaging_service.send_message("u6", "u7", "Mensaje adicional")

messaging_service.send_message("u7", "u8", "Hola u8")

messaging_service.send_message("u8", "u9", "Hola u9")
messaging_service.send_message("u8", "u9", "Mensaje adicional")

messaging_service.send_message("u9", "u10", "Hola u10")
print()

# Consumir mensajes
consume_message(rel_id, timeout) o cosume_by_users(sender_id, receiver_id, timeout) consume bloqueante en Redis e inserta en el histórico de mensajes de mongo

In [7]:
# Consumir mensajes entre u1 y u2
print("Consumiendo mensajes entre u1 y u2 ")

while True:
    msg = messaging_service.consume_by_users("u1", "u2")
    if msg is None:
        print("No quedan mensajes en la cola u1-u2")
        break

    print(msg)

Consumiendo mensajes entre u1 y u2 
MessageData(sender_id='u1', receiver_id='u2', content='Hola u2', rel_id='u1-u2', msg_id='d480bb24-13c0-421d-acab-0a33d25d22a8', timestamp=datetime.datetime(2026, 1, 11, 17, 47, 59, 883462, tzinfo=datetime.timezone.utc), metadata={})
MessageData(sender_id='u1', receiver_id='u2', content='¿Qué tal?', rel_id='u1-u2', msg_id='0fef0424-6447-48b8-970e-614699932298', timestamp=datetime.datetime(2026, 1, 11, 17, 47, 59, 889388, tzinfo=datetime.timezone.utc), metadata={})
MessageData(sender_id='u1', receiver_id='u2', content='Mensaje 3', rel_id='u1-u2', msg_id='4edb6234-d299-40a1-81cd-d1d344f09ee3', timestamp=datetime.datetime(2026, 1, 11, 17, 47, 59, 894116, tzinfo=datetime.timezone.utc), metadata={})
No quedan mensajes en la cola u1-u2


# Consultas
Comprobamos consultas de las distintas bases de datos

# NEO4J

In [8]:
print("Vecinos de u1 en el grafo:")
neo4j_api.neighbors("u1")

Vecinos de u1 en el grafo:


['u2', 'u5']

In [9]:
print("Vecinos de u1 ordenados por número de mensajes:")
neo4j_api.neighbors_ordered_by_messages("u1")

Vecinos de u1 ordenados por número de mensajes:


[('u2', 3), ('u5', 2)]

In [10]:
print("Camino óptimo entre u1 y u10 según número de mensajes:")
neo4j_api.best_path_by_messages("u1", "u10")

Camino óptimo entre u1 y u10 según número de mensajes:


{'nodes': ['u1', 'u5', 'u6', 'u7', 'u8', 'u9', 'u10'],
 'hops': 6,
 'cost': 2.333333333333333}

# MongoDB

In [11]:
# Mensajes enviados por u1 que han sido consumidos
print("Mensajes enviados por u1:")
mongo_api.count_messages_by_user("u1")

Mensajes enviados por u1:


3

In [12]:
# Mensajes enviados por u3 que han sido consumidos 
print("Mensajes enviados por u3:")
mongo_api.count_messages_by_user("u3")

Mensajes enviados por u3:


0

In [13]:
print("Mensajes enviados entre u1 y u2 entre 2024 y 2026:")
mongo_api.count_messages_by_time_for_relation(make_rel_id("u1", "u2"), start=datetime(2024, 1, 1), end=datetime(2026, 12, 31))

Mensajes enviados entre u1 y u2 entre 2024 y 2026:


[{'count': 3, 'periodo': '2026-01-11'}]

# Snapshots
Crear, obtener, listar, eliminar y comparar snapshots

# Primer Snapshot

In [14]:
# Snapshot inicial
name_a = "Snapshot A"
snapshot_id_a = snapshot_service.create_snapshot(name_a)
print(f"Snapshot '{name_a}' creado con ID: {snapshot_id_a}")

Snapshot 'Snapshot A' creado con ID: 6963e255501876e241dcb5d4


# Modificar estado actual
Añadir nuevos usuarios, relaciones y mensajes

In [15]:
new_users = [
    ("u11", "Kevin"),
    ("u12", "Laura"),
    ("u13", "Mario"),
]
for uid, nm in new_users:
    try:
        neo4j_api.create_user(uid, name=nm)
    except ValueError:
        pass # por si el usuario ya existe

new_rels = [
    ("u10", "u11"),
    ("u11", "u12"),
    ("u12", "u13"),
    ("u1", "u13"),
]
for a, b in new_rels:
    try:
        neo4j_api.create_relation(a, b)
    except ValueError:
        pass # por si la relación ya existe

more_messages = [
    ("u1", "u2", "Mensaje extra 1"),
    ("u1", "u2", "Mensaje extra 2"),
    ("u4", "u5", "Mensaje extra 3"),
    ("u10", "u11", "Bienvenido u11"),
    ("u11", "u12", "Hola u12"),
    ("u12", "u13", "Hola u13"),
    ("u1", "u13", "Hola nuevo u13"),
]

for s, r, c in more_messages:
    messaging_service.send_message(s, r, c)

# Consumir mensajes
consume_pairs = [
    ("u1", "u2"),
    ("u4", "u5"),
    ("u10", "u11"),
    ("u11", "u12"),
    ("u12", "u13"),
    ("u1", "u13"),
]
for s, r in consume_pairs:
    while True:
        msg = messaging_service.consume_by_users(s, r, timeout=1)
        if msg is None:
            break

# Segundo Snapshot

In [16]:
name_b = "Snapshot B"
snapshot_id_b = snapshot_service.create_snapshot(name_b)
print(f"Snapshot '{name_b}' creado con ID: {snapshot_id_b}")

Snapshot 'Snapshot B' creado con ID: 6963e25b501876e241dcb5df


# Comparar Snapshots
mongo_api.compare_snapshots(snapshot_name1, snapshot_name2) compara las estadísticas entre ambas snapshots. Imprime la diferencia *B - A*

In [17]:
comparison = mongo_api.compare_snapshots(name_a, name_b)
print("Comparación de snapshots (mensajes añadidos/eliminados):")
print(comparison)

Comparación de snapshots (mensajes añadidos/eliminados):
{'snapshot_a': {'name': 'Snapshot A', 'created_at': datetime.datetime(2026, 1, 11, 17, 48, 5, 142000), 'total_messages': 3, 'num_nodes': 10, 'num_relationships': 10, 'num_queued': 17}, 'snapshot_b': {'name': 'Snapshot B', 'created_at': datetime.datetime(2026, 1, 11, 17, 48, 11, 379000), 'total_messages': 13, 'num_nodes': 13, 'num_relationships': 14, 'num_queued': 14}, 'diff': {'total_messages': 10, 'num_nodes': 3, 'num_relationships': 4, 'num_queued': -3}}


# Restaurar Snapshot A

In [18]:
snapshot_service.restore_snapshot(snapshot_id_a)
print(f"Snapshot '{name_a}' restaurado.")

Snapshot 'Snapshot A' restaurado.


# Comprobaciones tras restore

In [19]:
# No debería existir el usuario u11
print("Usuario u11 existe después de restaurar snapshot A:")
u11 = neo4j_api.get_user("u11")
print("u11 =", u11)

Usuario u11 existe después de restaurar snapshot A:
u11 = None


In [20]:
# Mensajes en mongo tras restaurar snapshot A
print("Mensajes en MongoDB tras restaurar snapshot A:")
mongo_api.list_messages()

Mensajes en MongoDB tras restaurar snapshot A:


[{'_id': ObjectId('6963e25b501876e241dcb5e2'),
  'sender_id': 'u1',
  'receiver_id': 'u2',
  'content': 'Mensaje 3',
  'rel_id': 'u1-u2',
  'msg_id': '4edb6234-d299-40a1-81cd-d1d344f09ee3',
  'timestamp': datetime.datetime(2026, 1, 11, 17, 47, 59, 894000),
  'metadata': {}},
 {'_id': ObjectId('6963e25b501876e241dcb5e1'),
  'sender_id': 'u1',
  'receiver_id': 'u2',
  'content': '¿Qué tal?',
  'rel_id': 'u1-u2',
  'msg_id': '0fef0424-6447-48b8-970e-614699932298',
  'timestamp': datetime.datetime(2026, 1, 11, 17, 47, 59, 889000),
  'metadata': {}},
 {'_id': ObjectId('6963e25b501876e241dcb5e0'),
  'sender_id': 'u1',
  'receiver_id': 'u2',
  'content': 'Hola u2',
  'rel_id': 'u1-u2',
  'msg_id': 'd480bb24-13c0-421d-acab-0a33d25d22a8',
  'timestamp': datetime.datetime(2026, 1, 11, 17, 47, 59, 883000),
  'metadata': {}}]